# 🌌 Kuasar Simülasyonu

Bu proje, bir kara deliğin merkezinde bulunan **kuasar** yapısını 3D olarak modellemek ve animasyonunu oluşturmak amacıyla hazırlanmıştır.  
Kuasarlar, evrenin en parlak ve enerjik gök cisimleridir ve genellikle süper kütleli kara deliklerin etrafında oluşur.  
Merkezdeki kara delik, içine düşen maddeyi yutar, bu sırada **yoğun bir ışık ve parçacık fışkırması (jet)** meydana gelir.

---

## 🚀 Özellikler

- **Merkezde Süper Kütleli Kara Delik**: Tamamen siyah küre ile temsil edilir.
- **Olay Ufku ve Beyaz Kenar Çizgisi**: Kara deliğin sınırını vurgulamak için beyaz kontur çizgisi.
- **Akreasyon Diski**: Kara deliğin etrafında dönen sıcak gaz ve toz diskini simüle eden renkli halka.
- **Işık Bükülmesi Etkisi**: Diskin hem üst hem alt tarafı kapsayan eğilmiş yapısı.
- **Relativistik Jetler**: Kara deliğin kutuplarından çıkan parlak mavi jetler.
- **3D Animasyon**: Kara delik ve diskin dönme hareketi.

---

## 📦 Kullanılan Kütüphaneler

- `numpy` → Matematiksel hesaplamalar
- `matplotlib` → 3D görselleştirme ve animasyon
- `mpl_toolkits.mplot3d` → 3D eksen yönetimi

---

## ⚙ Çalışma Mantığı

1. **Merkez Küre (Kara Delik)**  
   - Siyah renkte, kenarsız bir küre çizilir.
   - Üzerine olay ufkunu göstermek için beyaz kontur eklenir.

2. **Akreasyon Diski**  
   - Renk geçişli (turuncu-sarı-beyaz) halka formunda çizilir.
   - Diskin üst ve alt tarafı hafif eğimle oluşturulur.

3. **Jetler**  
   - Kara deliğin kutuplarından çıkan mavi renkli koniler.
   - Yüksek hızda parçacık fışkırmasını temsil eder.

4. **Animasyon**  
   - Kamera açısı sürekli döndürülerek diskin dönme hareketi simüle edilir.

---

## ▶ Çalıştırma

Google Colab üzerinde çalıştırmak için:

```bash
pip install matplotlib numpy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import datetime

# ---------------------------
# Parametreler
# ---------------------------
r_s = 1.0              # Schwarzschild yarıçapı (fiziksel temel ölçek)
r_s_vis = 1.2          # Görsel olay ufku yarıçapı (biraz büyütülmüş)
disk_r_inner = r_s_vis * 1.4
disk_r_outer = r_s_vis * 3.2
orbit_radius = 5.0     # (isteğe bağlı obje için; burada jet odaklıyız)
time_step_sec = 10
frames = 360
interval = 30          # ms

# Jet parametreleri
N_particles = 250
jet_speed = 0.12       # parcacıkların bir frame’de aldığı mesafe
jet_radius = 0.25      # jetin yarıçapı (çekirdeğe yakın dar)
jet_helix_amp = 0.12   # helisel sapma genliği
jet_helix_freq = 3.0   # helisel frekans

# Disk çizimi için halka sayısı ve açısal örnekleme
N_rings = 40
N_theta = 240

# Görsel ışık rengi (matplotlib isimleri/colormapları)
disk_color = 'orange'
jet_base_color = np.array([1.0, 0.9, 0.3, 1.0])  # sarımsı-beyaz RGBA

# ---------------------------
# Zaman ve figür/eksen
# ---------------------------
earth_start = datetime.datetime(2025, 8, 12, 15, 26, 0)
earth_time = 0.0

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])
ax.set_zlim([-10, 10])
ax.axis('off')
fig.patch.set_facecolor('black')
ax.set_facecolor('black')

# Hafif arka plan “yıldız” efekti (statik scatter)
np.random.seed(42)
N_stars = 400
stars = (np.random.rand(N_stars, 3) - 0.5) * 2 * 20  # [-10,10] kutusunun biraz dışına
ax.scatter(stars[:,0], stars[:,1], stars[:,2], s=2, c='white', alpha=0.4, zorder=1)

# ---------------------------
# Olay ufku (kara delik)
# ---------------------------
u = np.linspace(0, 2*np.pi, 64)
v = np.linspace(0, np.pi, 32)
x_sphere = r_s_vis * np.outer(np.cos(u), np.sin(v))
y_sphere = r_s_vis * np.outer(np.sin(u), np.sin(v))
z_sphere = r_s_vis * np.outer(np.ones_like(u), np.cos(v))

# İç kısım: opak siyah
ax.plot_surface(
    x_sphere, y_sphere, z_sphere,
    color='black', edgecolor=None, alpha=1.0, zorder=30)

# Üstüne beyaz tel kafes (file ama seyrek ve ince)
ax.plot_wireframe(
    x_sphere, y_sphere, z_sphere,
    color='white', linewidth=0.6, rstride=6, cstride=6, zorder=35)

# Ekvator çizgisi (belli olsun)
theta_circle = np.linspace(0, 2*np.pi, 256)
ax.plot(r_s_vis*np.cos(theta_circle), r_s_vis*np.sin(theta_circle), 0*theta_circle,
        color='white', linewidth=1.4, zorder=40)

# ---------------------------
# Akresyon diski (parlak halka)
# ---------------------------
thetas = np.linspace(0, 2*np.pi, N_theta)
radii = np.linspace(disk_r_inner, disk_r_outer, N_rings)
z_wave_amp = 0.18

# Disk halkalarını çizip referansları tutalım (animasyonda renk/parlaklık güncelleyeceğiz)
disk_lines = []
for r in radii:
    # içe doğru parlaklık artışı + dalga
    # intensity ring bazlı hesap, frame’de flicker ile çarpacağız
    base_intensity = 0.55 + 0.45 * np.exp(-(r - disk_r_inner)*3.2)
    x_ring = r * np.cos(thetas)
    y_ring = r * np.sin(thetas)
    z_ring = z_wave_amp * np.sin(9*thetas + r*12.0)
    # Çizgi: alpha’yı intensity ile orantılı yap
    (line,) = ax.plot(x_ring, y_ring, z_ring,
                      color=disk_color, linewidth=2.6,
                      alpha=np.clip(base_intensity, 0.25, 1.0), zorder=25)
    disk_lines.append((line, base_intensity))

# Diskin iç kenarında ekstra parlak iç halka
inner_theta = np.linspace(0, 2*np.pi, 400)
x_in = disk_r_inner * np.cos(inner_theta)
y_in = disk_r_inner * np.sin(inner_theta)
z_in = 0.25 * np.sin(6*inner_theta + 3.0)
inner_line, = ax.plot(x_in, y_in, z_in, color='yellow', linewidth=2.0, alpha=0.6, zorder=28)

# ---------------------------
# Relativistik jetler (parçacıklar)
# ---------------------------
# Parçacıkları +z ve -z için karışık başlat
z_span = 9.0
z_positions = (np.random.rand(N_particles) - 0.5) * 2 * z_span
# Jet ekseni etrafında küçük rastgele açılar (radial pozisyon için)
phi = np.random.rand(N_particles) * 2*np.pi
r_rand = 0.35 * np.sqrt(np.random.rand(N_particles))  # merkez yakın yoğun
x0 = r_rand * np.cos(phi)
y0 = r_rand * np.sin(phi)

# Üst ve alt jeti ayırmak için maske
mask_up = z_positions >= 0
mask_dn = ~mask_up

# Jet renkleri (alpha’yı animasyonda güncelleyeceğiz)
jet_colors = np.tile(jet_base_color, (N_particles, 1))

jet_scatter = ax.scatter(x0, y0, z_positions, s=8, c=jet_colors, zorder=45)

# ---------------------------
# Metin ve kamera
# ---------------------------
time_text = fig.text(0.04, 0.95, '', color='white', fontsize=13)

# ---------------------------
# Yardımcı: disk flicker üreteci
# ---------------------------
def smooth_noise(n, smooth=0.9, seed=123):
    rng = np.random.default_rng(seed)
    x = rng.normal(size=n)
    y = np.zeros(n)
    acc = 0.0
    for i in range(n):
        acc = smooth*acc + (1-smooth)*x[i]
        y[i] = acc
    # 0..1 arası normalize et
    y -= y.min()
    if y.max() > 0: y /= y.max()
    return y

flicker = smooth_noise(frames, smooth=0.95, seed=999)

# ---------------------------
# Animasyon güncelleme
# ---------------------------
def update(frame):
    global earth_time
    earth_time += time_step_sec

    # 1) Kamera hafif döndür (azimuth)
    ax.view_init(elev=25, azim=(frame*0.7) % 360)

    # 2) Disk flicker (parlaklığı hafifçe çarp)
    fl = 0.85 + 0.35 * flicker[frame % frames]  # 0.85..1.2 civarı
    for line, base_intensity in disk_lines:
        alpha = np.clip(base_intensity * fl, 0.15, 1.0)
        line.set_alpha(alpha)
    inner_line.set_alpha(np.clip(0.5 * fl + 0.2, 0.2, 0.9))

    # 3) Jet parçacıklarını güncelle
    # z yönünde akış + helisel sapma
    # Yukarı gidenler:
    z_positions[mask_up] += jet_speed
    # Aşağı gidenler:
    z_positions[mask_dn] -= jet_speed

    # Sınır dışına çıkanları geri sar
    z_positions[z_positions > z_span] = 0.0
    z_positions[z_positions < -z_span] = 0.0

    # Helisel sapma: jet ekseninden açısal salınım
    helix_phase = frame / 10.0
    x = x0 + jet_helix_amp * np.sin(jet_helix_freq * (z_positions + helix_phase))
    y = y0 + jet_helix_amp * np.cos(jet_helix_freq * (z_positions + helix_phase))

    # Merkezde daha parlak, uçlara gidince sönüm (alpha) — basit bir profil
    # |z| büyüdükçe alpha azalsın:
    fade = 1.0 - (np.abs(z_positions) / z_span)
    fade = np.clip(fade, 0.05, 1.0)

    # Jet renklerini alpha ile güncelle
    cols = jet_colors.copy()
    cols[:, 3] = 0.25 + 0.55 * (fade * fl)  # 0.25..0.8 arası
    jet_scatter._offsets3d = (x, y, z_positions)
    jet_scatter.set_color(cols)

    # 4) Zaman yazısı
    earth_datetime = earth_start + datetime.timedelta(seconds=earth_time)
    time_text.set_text(f"Kuasar Simülasyonu  |  Zaman: {earth_datetime.strftime('%d/%m/%Y %H:%M:%S')}")

    return (jet_scatter, time_text, inner_line, *[dl for dl,_ in disk_lines])

anim = FuncAnimation(fig, update, frames=frames, interval=interval, blit=False)

from IPython.display import HTML
# Büyük animasyon uyarısı alırsan şunu kullanabilirsin:
# import matplotlib as mpl
# mpl.rcParams['animation.embed_limit'] = 50  # MB
HTML(anim.to_jshtml())
